# Compare Death Rate Trajectories
> Comparing different countries' death rate trajectories with those of Italy, South Korea, Japan and Lombardy, as well as US states.

- comments: false
- toc: true
- author: Pratap Vardhan, Vlad Stan
- categories: [growth, compare, death, interactive]
- image: images/covid-compare-country-death-trajectories.png
- permalink: /compare-country-death-trajectories/

In [1]:
#hide
import pandas as pd
import numpy as np
import altair as alt
from IPython.display import HTML

pd.options.mode.chained_assignment = None  # default='warn'

CHART_WIDTH = 600
CHART_HEIGHT = 450

In [2]:
#hide
url = ('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
df = pd.read_csv(url)
# rename countries
df['Country/Region'] = df['Country/Region'].replace({'Korea, South': 'South Korea'})
df = df[~df['Country/Region'].isin(['Cruise Ship'])]   # Remove Ships
df = df[~df['Country/Region'].isin(['Diamond Princess'])]   # Remove Diamond Princess
dt_cols = df.columns[~df.columns.isin(['Province/State', 'Country/Region', 'Lat', 'Long'])]

In [3]:
#hide
url_italy = ('https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv')
df_italy = pd.read_csv(url_italy)

In [4]:
#hide
dff = (df.groupby('Country/Region')[dt_cols].sum()
       .stack().reset_index(name='Confirmed Cases')
       .rename(columns={'level_1': 'Date', 'Country/Region': 'Country'}))
dff['Date'] = pd.to_datetime(dff['Date'], format='%m/%d/%y')

In [5]:
#hide
MIN_CASES = 100
LAST_DATE = dt_cols[-1]
# sometimes last column may be empty, then go backwards
for c in dt_cols[::-1]:
    if not df[c].fillna(0).eq(0).all():
        LAST_DATE = c
        break
countries = dff[dff['Date'].eq(LAST_DATE) & dff['Confirmed Cases'].ge(MIN_CASES) & 
        dff['Country'].ne('China')
       ].sort_values(by='Confirmed Cases', ascending=False)
countries = countries['Country'].values

# Make exception to MIN_CASES for Japan
countries = np.append(countries, ['Japan'])

In [6]:
#hide
SINCE_CASES_NUM = 10
COL_X = f'Days since {SINCE_CASES_NUM}th death'
dff2 = dff[dff['Country'].isin(countries)].copy()
days_since = (dff2.assign(F=dff2['Confirmed Cases'].ge(SINCE_CASES_NUM))
              .set_index('Date')
              .groupby('Country')['F'].transform('idxmax'))
dff2[COL_X] = (dff2['Date'] - days_since.values).dt.days.values
dff2 = dff2[dff2[COL_X].ge(0)]

In [7]:
#hide
# more regions of Italy can be added alongside Lombardia in the next line of code (ex: 'Emilia Romagna', 'Veneto', etc.)
df_reg = df_italy[df_italy['denominazione_regione'].isin(['Lombardia'])]
df_reg = df_reg.sort_values(by=['denominazione_regione', 'deceduti'])

# Keep only regions that have at least MIN_CASES deaths, and keep data starting from the SINCE_CASES_NUM death
regions = df_reg[df_reg['deceduti'].ge(MIN_CASES)]
regions = regions['denominazione_regione'].values
df_reg2 = df_reg[df_reg['denominazione_regione'].isin(regions)].copy()
df_reg2 = df_reg2[df_reg2['deceduti'].ge(SINCE_CASES_NUM)]

df_simple = df_reg2[['denominazione_regione','data', 'deceduti']]
df_simple.rename(columns={'denominazione_regione':'Country'}, inplace=True)
df_simple.rename(columns={'data':'Date'}, inplace=True)
df_simple.rename(columns={'deceduti':'Confirmed Cases'}, inplace=True)
df_simple['Date'] = pd.to_datetime(df_simple["Date"]).dt.strftime('%Y-%m-%d')
df_simple['Date'] = pd.to_datetime(df_simple["Date"])

In [8]:
#hide
df_simple[f'Days since {SINCE_CASES_NUM}th death'] = ''
current_region = df_simple.iloc[0]['Country']
days = 0
for index, row in df_simple.iterrows():
  if row['Country'] == current_region:
    df_simple[f'Days since {SINCE_CASES_NUM}th death'][index] = days
    days += 1
  else:
    days = 0
    current_region = df_simple['Country'][index]
    df_simple[f'Days since {SINCE_CASES_NUM}th death'][index] = days
    days += 1

df_simple["Country"]= df_simple["Country"].replace("Lombardia", "Lombardy")

dff2 = dff2.append(df_simple) # append Italy data to the rest of the data

In [9]:
#hide
def get_country_colors(x):
    mapping = {
        'Italy': 'black',
        'Lombardy': '#DC143C',
        'Iran': '#A1BA59',
        'South Korea': '#C71585',
        'Spain': 'green',
        'Germany': '#4DC6BD',
        'France': 'purple',
        'US': '#0059B3',
        'Switzerland': '#ff0000',
        'Norway': '#C1B7AD',
        'United Kingdom': '#2495D3',
        'Netherlands': 'orange',
        'Sweden': '#FFCD00',
        'Belgium': '#6f0000',
        'Denmark': '#C60C30',
        'Austria': '#ED2939',
        'Portugal': '#006600',
        'Romania': '#FF4500',
        'Indonesia': '#FF6666',
        'Canada': '#990000',
        'Turkey': '#7499c7',
        'Brazil': 'mediumseagreen',
        'Japan': '#9467bd'}
    return mapping.get(x, '#C1B7AD')

In [10]:
#hide_input
baseline_countries = ['Italy', 'South Korea', 'Japan']
max_date = dff2['Date'].max()
color_domain = list(dff2['Country'].unique())
color_range = list(map(get_country_colors, color_domain))

def make_since_chart(highlight_countries=[], baseline_countries=baseline_countries):
    selection = alt.selection_multi(fields=['Country'], bind='legend', 
                                    init=[{'Country': x} for x in highlight_countries + baseline_countries])

    base = alt.Chart(dff2, width=CHART_WIDTH, height=CHART_HEIGHT).encode(
        x=f'{COL_X}:Q',
        y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cumulative Deaths')),
        color=alt.Color(
            'Country:N',
            scale=alt.Scale(domain=color_domain, range=color_range),
            legend=alt.Legend(columns=2, symbolLimit=100)),
        tooltip=list(dff2),
        opacity=alt.condition(selection, alt.value(1), alt.value(0.05))
    )
    max_day = dff2[COL_X].max()
    ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'Confirmed Cases'])
    base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='Confirmed Cases:Q')
    return (
        base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) +
        base_ref.transform_filter(
            alt.datum[COL_X] >= max_day
        ).mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth') +
        base.mark_line(point=True).add_selection(selection) + 
        base.transform_filter(
            alt.datum['Date'] >= int(max_date.timestamp() * 1000)
        ).mark_text(dy=-8, align='right', fontWeight='bold').encode(text='Country:N')
    ).properties(
        title=f"Compare {', '.join(highlight_countries)} death trajectory with {', '.join(baseline_countries)}"
    )

## Learning from Italy, South Korea, Japan

Italy, South Korea & Japan are three countries which show significantly different total deaths growth rates and evolution over time. 

**South Korea** and **Japan** have a lower growth rate since thier 10th death. **Italy** has been growing close to 33% until 3 weeks since their 10th death.

Where does your country stand today?

Click (Shift+ for multiple) on Countries legend to filter the visualization.

In [11]:
#hide_input
HTML(f'<small class="float-right">Last Updated on {pd.to_datetime(LAST_DATE).strftime("%B, %d %Y")}</small>')

In [12]:
#hide_input
chart = make_since_chart()
chart

alt.LayerChart(...)

In [13]:
#hide_input
chart2 = make_since_chart(['Spain', 'Germany'])
chart2

alt.LayerChart(...)

In [14]:
#hide_input
chart2 = make_since_chart(['Lombardy', 'Netherlands', 'US', 'United Kingdom', 'Spain'])
chart2

alt.LayerChart(...)

In [15]:
#hide_input
chart3 = make_since_chart(['US', 'France'])
chart3

alt.LayerChart(...)

In [16]:
#hide_input
chart4 = make_since_chart(['Germany', 'United Kingdom'])
chart4

alt.LayerChart(...)

Select a country from the drop down list below to toggle  the visualization.

In [17]:
#hide_input
base = alt.Chart(dff2, width=CHART_WIDTH, height=CHART_HEIGHT).encode(
    x=f'{COL_X}:Q',
    y=alt.Y('Confirmed Cases:Q', scale=alt.Scale(type='log'), axis=alt.Axis(title='Cumulative Deaths')),
    color=alt.Color('Country:N', scale=alt.Scale(domain=color_domain, range=color_range), legend=None),
    tooltip=['Country', 'Date', 'Confirmed Cases', COL_X]
)

country_selection = alt.selection_single(
    name='Select', fields=['Country'], 
    bind=alt.binding_select(options=list(sorted(set(countries) - set(baseline_countries)))),
    init={'Country': 'US'})

date_filter = alt.datum['Date'] >= int(max_date.timestamp() * 1000)
base2 = base.transform_filter(alt.FieldOneOfPredicate('Country', baseline_countries))
base3 = base.transform_filter(country_selection)
base4 = base3.transform_filter(date_filter)

max_day = dff2[COL_X].max()
ref = pd.DataFrame([[x, SINCE_CASES_NUM*1.33**x] for x in range(max_day+1)], columns=[COL_X, 'Confirmed Cases'])
base_ref = alt.Chart(ref).encode(x=f'{COL_X}:Q', y='Confirmed Cases:Q')
base_ref_f = base_ref.transform_filter(alt.datum[COL_X] >= max_day)

chart5 = (
 base_ref.mark_line(color='black', opacity=.5, strokeDash=[3,3]) + 
 base_ref_f.mark_text(dy=-6, align='right', fontSize=10, text='33% Daily Growth') + 
 base2.mark_line(point=True, tooltip=True) +
 base3.mark_line(point={'size':50}, tooltip=True) +
 base2.transform_filter(date_filter).mark_text(dy=-8, align='right').encode(text='Country:N') +
 base4.mark_text(dx=8, align='left', fontWeight='bold').encode(text='Country:N') +
 base4.mark_text(dx=8, dy=12, align='left', fontWeight='bold').encode(text='Confirmed Cases:Q')
).add_selection(country_selection).properties(
    title=f"Country's death trajectory compared to {', '.join(baseline_countries)}"
)
chart5

alt.LayerChart(...)

## Learning From Lombardy, Italy

Lombardy, a region of Italy with a population of only 10 million people, has reached over 8,000 deaths due to Covid-19. At its highest daily death toll, Lombardy had 546 deaths in one day. This shows that the virus has a high mortality rate even on a small demographic scale, likely due to overwhelmed healthcare services.

Where does your country stand in comparison to Lombardy?  

Click (Shift+ for multiple) on Countries legend to filter the visualization.

In [18]:
#hide_input
chart5 = make_since_chart(['United Kingdom', 'US', 'Spain', 'Netherlands', 'France', 'South Korea'], ['Lombardy'])
chart5

alt.LayerChart(...)

Visualizations are created by [Pratap Vardhan](https://twitter.com/PratapVardhan)[^1]

Lombardy comparison chart by [Vlad Stan](https://twitter.com/vladpke). 

[^1]: Source: ["2019 Novel Coronavirus COVID-19 (2019-nCoV) Data Repository by Johns Hopkins CSSE"](https://systems.jhu.edu/research/public-health/ncov/) [GitHub repository](https://github.com/CSSEGISandData/COVID-19). Link to [original notebook](https://github.com/pratapvardhan/notebooks/blob/master/covid19/covid19-compare-country-death-trajectories.ipynb).

[^2]: Source: ["Dati COVID-19 Italia"](http://www.salute.gov.it/portale/home.html) [GitHub repository](https://github.com/pcm-dpc/COVID-19).